# Document Similarity & Topic Modelling

## Document Similarity


In [1]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    

    
    
    tokens = nltk.word_tokenize(doc)
    pos_tag = nltk.pos_tag(tokens)
    tags = [tag[1] for tag in pos_tag]
    c_tag = [convert_tag(tag) for tag in tags]
    res = list(zip(tokens,c_tag))
    sets = [wn.synsets(x,y) for x,y in res]
    
    return [val[0] for val in sets if len(val) > 0]# Your Answer Here


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    
    
    sem=[]
    for i1 in s1:
        r=[]
        scores=[x for x in [i1.path_similarity(i2) for i2 in s2]if x is not None]
        if scores:
            sem.append(max(scores))
    
    sc = sum(sem)/len(sem)
    
    return sc


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [2]:
document_path_similarity('I read a book', 'Can i read a book')

0.7763888888888889

### test_document_path_similarity

In [3]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

In [4]:
test_document_path_similarity()

0.554265873015873

In [5]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


___

### most_similar_docs

In [6]:
def most_similar_docs():
    
    
    similarities = [(paraphrase['D1'], paraphrase['D2'], document_path_similarity(paraphrase['D1'], paraphrase['D2']))
                    for index, paraphrase in paraphrases.iterrows()]
    
    return max(similarities, key=lambda item:item[2])

In [7]:
most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9753086419753086)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [8]:
from sklearn.metrics import accuracy_score

In [9]:
def add_similarity_labels(sample):
    sample['similarity_score'] = document_path_similarity(sample['D1'], sample['D2'])
    sample['paraphrase'] = 1 if sample['similarity_score'] > 0.75 else 0
    return sample

In [10]:
def label_accuracy():
    from sklearn.metrics import accuracy_score

    
    df = paraphrases.apply(add_similarity_labels, axis=1)
    ac_score = accuracy_score(df['Quality'].tolist(), df['paraphrase'].tolist())
    return ac_score

In [11]:
label_accuracy()

0.80000000000000004

## Topic Modelling

Lets use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [12]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [13]:
len(corpus)

2000

In [14]:
len(id_map)

901

In [15]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`


ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word=id_map, num_topics=10, passes=3, random_state=32)

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function should return a list of tuples.*

In [16]:
def lda_topics():
   
    return ldamodel.print_topics(num_topics=10, num_words=10) 

In [17]:
lda_topics()

[(0,
  '0.019*"year" + 0.015*"don" + 0.015*"team" + 0.011*"think" + 0.011*"time" + 0.010*"good" + 0.010*"know" + 0.009*"new" + 0.009*"just" + 0.008*"better"'),
 (1,
  '0.022*"drive" + 0.014*"disk" + 0.012*"know" + 0.012*"hard" + 0.011*"card" + 0.011*"drives" + 0.010*"does" + 0.010*"just" + 0.009*"controller" + 0.009*"people"'),
 (2,
  '0.022*"thanks" + 0.014*"research" + 0.013*"medical" + 0.013*"center" + 0.013*"university" + 0.011*"new" + 0.009*"good" + 0.009*"know" + 0.008*"advance" + 0.008*"edu"'),
 (3,
  '0.021*"game" + 0.015*"don" + 0.014*"just" + 0.011*"games" + 0.011*"time" + 0.010*"think" + 0.008*"hit" + 0.007*"way" + 0.007*"got" + 0.007*"period"'),
 (4,
  '0.036*"space" + 0.019*"nasa" + 0.012*"information" + 0.012*"god" + 0.011*"data" + 0.010*"use" + 0.010*"available" + 0.010*"edu" + 0.010*"atheism" + 0.008*"ftp"'),
 (5,
  '0.023*"car" + 0.011*"know" + 0.009*"000" + 0.009*"pick" + 0.009*"don" + 0.008*"time" + 0.008*"new" + 0.008*"lot" + 0.007*"april" + 0.007*"ground"'),
 (6,
 

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [18]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [19]:
def topic_distribution():
    
    
    new_doc_transformed = vect.transform(new_doc)
    corpus = gensim.matutils.Sparse2Corpus(new_doc_transformed, documents_columns=False)
    doc_topics = ldamodel.get_document_topics(corpus)
    topic_dist = []
    for val in list(doc_topics):
        for v in val:
            topic_dist.append(v)
    return topic_dist

In [20]:
topic_distribution()

[(0, 0.020006255514384074),
 (1, 0.020002293869384392),
 (2, 0.020000292321516056),
 (3, 0.020001923096439871),
 (4, 0.81996800290548211),
 (5, 0.020003015286586261),
 (6, 0.020003161318757269),
 (7, 0.020011045596764386),
 (8, 0.020002029268088251),
 (9, 0.020001980822597215)]